## The program takes a markdown table output (from CHATGPT in a separate browser) then converts to a usable table.
#### Example of a markdown text input below (toggle on configure to see it).

| Table Name (UI) | Technical Name | Description |
| --- | --- | --- |
| Business Unit | cmn_cost_center | Contains records for each business unit within your organization. Business units can be used for categorizing costs, work, and other elements at a more granular level than a company. |
| Department | cmn_department | Contains records for each department within your organization. Each user can be assigned to a department, which can be used for routing decisions, reporting, and other functions. |
| Location | cmn_location | Contains records for physical locations where your company operates. This can include offices, data centers, or other locations. Each user and asset can be associated with a location. |
| Company | core_company | Contains records for each company that your organization interacts with. This can include your own company, vendors, partners, and customers. |
| User | sys_user | Contains a record for each individual who has (or had) access to your ServiceNow instance. It includes information like their name, email, roles, etc. |
| Group | sys_user_group | Contains records for each group of users within your ServiceNow instance. Groups can be used for many purposes, like assignment, approval rules, notifications, and access control. |
| Group Members | sys_user_grmember | Contains records of the relationships between users and groups. Each record represents a user being a member of a group. |
| Role | sys_user_role | Contains records for each role within your ServiceNow instance. Roles are used to define what actions a user can perform and what data they can access. |

In [4]:
import pandas as pd
from io import StringIO
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

markdown_table = """
| Table Name (UI) | Technical Name | Description |
| --- | --- | --- |
| Business Unit | cmn_cost_center | Contains records for each business unit within your organization. Business units can be used for categorizing costs, work, and other elements at a more granular level than a company. |
| Department | cmn_department | Contains records for each department within your organization. Each user can be assigned to a department, which can be used for routing decisions, reporting, and other functions. |
| Location | cmn_location | Contains records for physical locations where your company operates. This can include offices, data centers, or other locations. Each user and asset can be associated with a location. |
| Company | core_company | Contains records for each company that your organization interacts with. This can include your own company, vendors, partners, and customers. |
| Vendor | core_company | Contains records for each vendor that your organization interacts with. In ServiceNow, vendors are often stored in the Company table, and differentiated by a field such as type or category. |
| User | sys_user | Contains a record for each individual who has (or had) access to your ServiceNow instance. It includes information like their name, email, roles, etc. |
| Group | sys_user_group | Contains records for each group of users within your ServiceNow instance. Groups can be used for many purposes, like assignment, approval rules, notifications, and access control. |
| Group Members | sys_user_grmember | Contains records of the relationships between users and groups. Each record represents a user being a member of a group. |
| Role | sys_user_role | Contains records for each role within your ServiceNow instance. Roles are used to define what actions a user can perform and what data they can access. |
"""

# Split markdown table into lines
markdown_lines = markdown_table.split("\n")[3:-1]

# Create a pandas DataFrame
dataframe = pd.read_csv(StringIO('\n'.join(markdown_lines)), sep='|', header=None)

# Remove white spaces
dataframe = dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Set column names manually
dataframe.columns = ['Unnamed: 0', 'Table Name (UI)', 'Technical Name', 'Description', 'Unnamed: 4']

# Drop empty columns
dataframe = dataframe.dropna(axis=1)

# Create a workbook and add a worksheet to it
wb = Workbook()
ws = wb.active

# Define fills for cells
header_fill = PatternFill(start_color="182546",
                           end_color="182546",
                           fill_type="solid")

# Define fonts for cells
header_font = Font(color="FFFFFF", bold=True)
default_font = Font(color="000000")

# Define cell alignment
alignment = Alignment(horizontal='left', vertical='center', wrap_text=True)

# Add data to worksheet
for i, column in enumerate(dataframe.columns):
    cell = ws.cell(row=1, column=i+1)
    cell.value = column
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = alignment

for r, row in enumerate(dataframe_to_rows(dataframe, index=False, header=False), start=2):
    for c, value in enumerate(row, start=1):
        cell = ws.cell(row=r, column=c, value=value)
        cell.font = default_font
        cell.alignment = alignment

# Autofit column widths
for column_cells in ws.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    column_cells[0].parent.column_dimensions[column_cells[0].column_letter].width = length

# Save the workbook
wb.save("itsm_foundation_data_tables.xlsx")


Less efficient method
* Convert to dataframe
* Convert to html (then open in browser)

In [5]:
# import pandas as pd
# from io import StringIO
# from IPython.display import display, HTML

# markdown_table = """
# | Table Name (UI) | Technical Name | Description |
# | --- | --- | --- |
# | Business Unit | cmn_cost_center | Contains records for each business unit within your organization. Business units can be used for categorizing costs, work, and other elements at a more granular level than a company. |
# | Department | cmn_department | Contains records for each department within your organization. Each user can be assigned to a department, which can be used for routing decisions, reporting, and other functions. |
# | Location | cmn_location | Contains records for physical locations where your company operates. This can include offices, data centers, or other locations. Each user and asset can be associated with a location. |
# | Company | core_company | Contains records for each company that your organization interacts with. This can include your own company, vendors, partners, and customers. |
# | User | sys_user | Contains a record for each individual who has (or had) access to your ServiceNow instance. It includes information like their name, email, roles, etc. |
# | Group | sys_user_group | Contains records for each group of users within your ServiceNow instance. Groups can be used for many purposes, like assignment, approval rules, notifications, and access control. |
# | Group Members | sys_user_grmember | Contains records of the relationships between users and groups. Each record represents a user being a member of a group. |
# | Role | sys_user_role | Contains records for each role within your ServiceNow instance. Roles are used to define what actions a user can perform and what data they can access. |
# """

# # Split markdown table into lines
# markdown_lines = markdown_table.split("\n")[3:-1]

# # Create a pandas DataFrame
# dataframe = pd.read_csv(StringIO('\n'.join(markdown_lines)), sep='|', header=None)

# # Remove white spaces
# dataframe = dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# # Set column names manually
# dataframe.columns = ['Unnamed: 0', 'Table Name (UI)', 'Technical Name', 'Description', 'Unnamed: 4']

# # Drop empty columns
# dataframe = dataframe.dropna(axis=1)

# dataframe

# # # Convert the DataFrame to HTML and display
# # html_table = dataframe.to_html(index=False)
# # display(HTML(html_table))


In [6]:
# # Save the DataFrame to HTML and save as a file
# html_table = dataframe.to_html(index=False)
# with open('itsm_foundation_data_table.html', 'w') as f:
#     f.write(html_table)
